In [182]:
import numpy as np
from numpy import sqrt, exp

def mc_heston(option_type, S0, k, T, initial_var, long_term_var, rate_reversion, vol_of_vol, corr, r, num_reps, steps):
    """
    option_type: "p" for put, "c" for call
    S0: the spot price of underlying stock
    K: the strike price
    T: the maturity of options
    initial_var: the initial value of variance
    long_term_var: the long term average price of variance
    rate_reversion: the mean reversion rate for the variance
    vol_of_vol: the volatility of volatility
    corr: the correlation between the standard normal randon variable W1 & W2
    r: the risk free rate
    reps: the number of repeat monte carlo simulation
    steps: the number of steps in each simulation
    """
    delta_t = 1/float(steps)
    payoff = 0
    for i in range(steps):
        vt = initial_var
        st = S0
        for j in range(steps):
            w1 = np.random.normal(0,1)
            w2 = corr*w1+sqrt(1-corr**2)*np.random.normal(0,1)
            vt = (sqrt(vt)+0.5*vol_of_vol*sqrt(delta_t)*w1)**2 - rate_reversion*(vt-long_term_var)*delta_t - 0.25*vol_of_vol**2*delta_t
            st = st*exp((r-0.5*vt)*delta_t + sqrt(vt*delta_t)*w2)
        if option_type == "c":
            payoff += max(st-k,0)
        elif option_type == "p":
            payoff += max(k-st,0)
    return (payoff/num_reps)*(exp(-r*T))

In [92]:
import pandas as pd
from numpy import sqrt, mean, log, diff
import QuantLib as ql
from pandas_datareader.data import Options
import pandas_datareader.data as web
import datetime

In [93]:
SPY = web.YahooOptions('SPY')

In [94]:
for exp in SPY.expiry_dates:
    print(exp.isoformat())

2021-07-06
2021-07-07
2021-07-09
2021-07-12
2021-07-14
2021-07-16
2021-07-19
2021-07-21
2021-07-23
2021-07-26
2021-07-28
2021-07-30
2021-08-02
2021-08-04
2021-08-06
2021-08-20
2021-09-17
2021-09-30
2021-10-15
2021-11-19
2021-12-17
2021-12-31
2022-01-21
2022-02-18
2022-03-18
2022-03-31
2022-06-17
2022-09-16
2022-12-16
2023-01-20
2023-03-17
2023-12-15


In [98]:
spy_oct_exp = SPY.get_call_data(month=10,year=2021)
spy_oct_exp.head()

,,,,Last,Bid,Ask,Chg,PctChg,Vol,Open_Int,IV,Root,IsNonstandard,Underlying,Underlying_Price,Quote_Time,Last_Trade_Date,JSON
Strike,Expiry,Type,Symbol,,,,,,,,,,,,,,,
190.0,2021-10-15,call,SPY211015C00190000,219.60,224.79,225.29,0.0,0.0,1.0,0.0,0.000010,SPY,False,SPY,433.72,2021-07-02 20:00:02,2021-05-19 19:28:00,"{'contractSymbol': 'SPY211015C00190000', 'stri..."
195.0,2021-10-15,call,SPY211015C00195000,185.09,205.29,207.39,0.0,0.0,3.0,0.0,0.000010,SPY,False,SPY,433.72,2021-07-02 20:00:02,2021-03-05 18:15:59,"{'contractSymbol': 'SPY211015C00195000', 'stri..."
200.0,2021-10-15,call,SPY211015C00200000,226.45,233.19,234.18,0.0,0.0,1.0,2.0,0.647464,SPY,False,SPY,433.72,2021-07-02 20:00:02,2021-06-25 16:16:01,"{'contractSymbol': 'SPY211015C00200000', 'stri..."
210.0,2021-10-15,call,SPY211015C00210000,205.07,205.02,206.10,0.0,0.0,NaN,1.0,0.000010,SPY,False,SPY,433.72,2021-07-02 20:00:02,2021-04-21 17:21:47,"{'contractSymbol': 'SPY211015C00210000', 'stri..."
215.0,2021-10-15,call,SPY211015C00215000,171.93,0.00,0.00,0.0,0.0,NaN,0.0,0.000010,SPY,False,SPY,433.72,2021-07-02 20:00:02,2021-03-15 04:04:51,"{'contractSymbol': 'SPY211015C00215000', 'stri..."


In [170]:
strikes = list(spy_oct_exp.index.get_level_values('Strike'))
IV = list(spy_oct_exp['IV'])
combined = pd.DataFrame(list(zip(strikes,IV)),columns=['Strikes','IV'])
spot_strike = combined[combined['Strikes'] == 430.0].index[0]
final_strike = [400.0,405.0,410.0,415.0,420.0,425.0,430.0,435.0,440.0,445.0,450.0,455.0,460.0]
final_IV = []
for i in range(len(final_strike)):
    col_num = combined[combined['Strikes'] == final_strike[i]].index[0]
    final_IV.append(combined.loc[col_num]['IV'])
new_data = pd.DataFrame(list(zip(final_strike,final_IV)),columns=['Strikes','IV'])

In [100]:
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()
calculation_date = ql.Date(3,7,2021)
expiry = ql.Date(15,10,2021)
spot = spy_oct_exp['Underlying_Price'].iloc[0]

In [101]:
ql.Settings.instance().evaluationDate = calculation_date
dividend_yield = ql.QuoteHandle(ql.SimpleQuote(0.0))
risk_free_rate = 0.0
dividend_rate = 0.0

In [102]:
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,risk_free_rate,day_count))
dividend_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))

In [103]:
# Dummy Paramters
initial_var = 0.2
rate_reversion = 0.5
long_term_var = 0.2
corr = -0.75
vol_of_vol = 0.2

In [104]:
process = ql.HestonProcess(flat_ts,dividend_ts,ql.QuoteHandle(ql.SimpleQuote(spot)),
                           initial_var, rate_reversion, long_term_var, vol_of_vol, corr)
model = ql.HestonModel(process)
engine = ql.AnalyticHestonEngine(model)

In [171]:
heston_helpers = []
date = expiry
for j, s in enumerate(final_strike):
    t = (date-calculation_date)
    p = ql.Period(t, ql.Days)
    sigma = final_IV[j]
    helper = ql.HestonModelHelper(p, calendar, spot, s, ql.QuoteHandle(ql.SimpleQuote(sigma)), flat_ts, dividend_ts)
    helper.setPricingEngine(engine)
    heston_helpers.append(helper)

In [172]:
lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
model.calibrate(heston_helpers, lm, ql.EndCriteria(500, 50, 1.0e-8,1.0e-8, 1.0e-8))
long_term_var, rate_reversion, vol_of_vol, corr, initial_var = model.params()
print("long_term_var = %f, rate_reversion = %f, vol_of_vol = %f, corr = %f, initial_var = %f" % (long_term_var, rate_reversion, vol_of_vol, corr, initial_var))

long_term_var = 0.195234, rate_reversion = 0.000002, vol_of_vol = 0.218181, corr = -0.836103, initial_var = 0.199177


In [183]:
#mc_heston(opton_type, S0, K, T, initial_var, long_term_var, rate_reversion, vol_of_vol, cprr, r, num_reps, steps):
mc_heston("c",433.72,430.0,expiry,initial_var,long_term_var,rate_reversion,vol_of_vol,corr,risk_free_rate,1000,1) 

TypeError: unsupported operand type(s) for *: 'float' and 'Date'